# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yma80\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
# load data from database
def load_data(db_filepath='DisasterResponse.db', sample_frac=1.0):
    """Load the data from database.
    Args:
        db_filepath: str
            Filepath to the SQL database.
        sample_frac: int
            Apply random subsampling and select only frac of entries.
    Returns:
        X, Y: np.ndarray or pandas.DataFrame
            Matrices of predictors (messages) and responses.
    """
    
    engine = create_engine('sqlite:///'+db_filepath)
    df = pd.read_sql_table('merged', engine).sample(frac=sample_frac, random_state=42)
    X = df['message']
#     Y = df[['related', 'request', 'offer', 'aid_related', 'infrastructure_related', 'weather_related']]
#     Y = df[ [c for c in df.columns if 'related' in c] ]
    Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original', 'genre', 'child_alone')] ]
#     Y = Y.replace({'genre':{'direct':0, 'news':1, 'social':2}})

    category_names = Y.columns

    return X, Y, category_names

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('merged', engine).sample(frac=1.0, random_state=42)
X = df['message']
Y = df[ [c for c in df.columns if c not in ('id', 'message', 'original')] ]
df.describe()

df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [252]:
# Investigate the hierarchy of responses
df['req_or_off'] = df.request | df.offer
df['any_related'] = df.aid_related | df.infrastructure_related | df.weather_related

df1 = df.query("related == 1")

df1.query("req_or_off==0")[['req_or_off', 'direct_report']+[c for c in df.columns if 'related' in c]].describe()

df1.query("offer==1").message

11096    I have many towels ( large , as well as hand t...
11293    When and Where should hot meals be delivered t...
11335    We did a large donation drive in our community...
75             I am a driver, a mechanic ,. I want to help
23986    Following the attack on Ngouboua village by ar...
10646                    Clothes sorting . Giving out food
10854    I can bring batteries , toiletries , non-peris...
11202    Looking to donate new ( several-year shelf lif...
25301    The returnees were provided with UNHCR's stand...
18098    - UNFPA, WHO and UNICEF are building over 100 ...
10992    I 'm a nutritionist and would be also happy to...
10607    i 'm also good with food prep if there 's a ne...
21684    UNICEF has already sent supplies to affected a...
11187    Will provide & serve nonperishable food . Also...
25201    MSF teams are active with one Zodiac (inflatab...
25291    World Bank Country Director Xian Zhu told a pr...
18008    This assistance, valued at $200.7 million, was.

### 2. Write a tokenization function to process your text data

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### 3.1 Classify Related/Unrelated messages

In [196]:
X, Y, cat_names = load_data(sample_frac=0.1)
X_train_txt, X_test_txt, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.25)

In [163]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
class DenseTransformer(TransformerMixin):
    """Transforms a sparse matrix to its dense representation."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()
    
pl_TextVectorise = Pipeline([
                        ('vectr', CountVectorizer(tokenizer=tokenize, max_features=1500)),
                        ('tfidf', TfidfTransformer()),
#                         ('dense', DenseTransformer()),
#                         ('scale', StandardScaler())
                ])

pl_FullNLP = FeatureUnion([
                            ('txtvec', pl_TextVectorise),
                            ('length', LengthEstimator())
                    ])

X_train = pl_FullNLP.fit_transform(X_train_txt)
X_test = pl_FullNLP.transform(X_test_txt)

KeyboardInterrupt: 

In [40]:
X_test.shape

(3273, 1501)

In [189]:
class LengthEstimator(BaseEstimator, TransformerMixin):
    """A custom transformer class to estimate the length of a message."""

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return pd.DataFrame(pd.Series(X).str.len())

def tokenize(text):
    """Turn a text string into an array of words.
        Args:
            text: str
        Returns:
            A list of lemmatised tokens (words).
    """
    
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    
    text = nltk.word_tokenize(text)
    
    text = [w for w in text if w not in stopwords.words('english')]
    
    # Reduce words to their root form
    text = [WordNetLemmatizer().lemmatize(w, pos='v') for w in text]
    
    return text

pipeline = build_model()

# train classifier
pipeline.fit(X_train_txt, Y_train)

# predict on test data
Y_pred = pipeline.predict(X_test_txt)

Y_pred = pd.DataFrame(Y_pred, columns = Y_test.columns)
Y_pred.describe()

Exception ignored in: <bound method SeekableUnicodeStreamReader.__del__ of <nltk.data.SeekableUnicodeStreamReader object at 0x0000018CBF6C5668>>
Traceback (most recent call last):
  File "C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\nltk\data.py", line 1160, in __del__
    self.close()
  File "C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\nltk\data.py", line 1189, in close
    self.stream.close()
KeyboardInterrupt


KeyboardInterrupt: 

In [198]:
# print(pipeline.get_params())
def build_model():
    """Define and train the classification pipeline.

        The pipeline takes as its input a string representing a text message,
        transforms it into TF-IDF vector, and runs a multi-target classification
        algorithm with 36 targets.
                
    """

    # A (sub-)pipeline to vectorize the message and ombine it with a text-length feature
    pl_NLP = FeatureUnion([
                    ('pl_Txt', Pipeline([
                                        ('vectr', CountVectorizer(tokenizer=tokenize, max_features=1500)),
                                        ('tfidf', TfidfTransformer()),
                                ]) ),
                    ('length', LengthEstimator())
                ])

    # Add an SVC classifier
    pl_Final = Pipeline([
                    ('pl_NLP', pl_NLP),
                    ('clsfr', MultiOutputClassifier(SVC(gamma='auto')))
               ])
    
    # Run GridSearch to optimize some of the parameters
    parameters = {'pl_NLP__pl_Txt__vectr__max_df' : [1.0, 0.9],
              'clsfr__estimator__kernel' : ['rbf', 'poly']}

    cv = GridSearchCV(pl_Final, param_grid=parameters, cv=3, verbose=4, scoring='f1_weighted')

    return cv

model = build_model()

model.fit(X_train_txt, Y_train)


# model = cv.best_estimator_

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clsfr__estimator__kernel=rbf, pl_NLP__pl_Txt__vectr__max_df=1.0 .


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



[CV]  clsfr__estimator__kernel=rbf, pl_NLP__pl_Txt__vectr__max_df=1.0, score=0.25690278937897565, total=  23.6s
[CV] clsfr__estimator__kernel=rbf, pl_NLP__pl_Txt__vectr__max_df=1.0 .


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.9s remaining:    0.0s
C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



[CV]  clsfr__estimator__kernel=rbf, pl_NLP__pl_Txt__vectr__max_df=1.0, score=0.2645870464297415, total=  20.0s
[CV] clsfr__estimator__kernel=rbf, pl_NLP__pl_Txt__vectr__max_df=1.0 .


...

[CV]  clsfr__estimator__kernel=poly, pl_NLP__pl_Txt__vectr__max_df=0.9, score=0.37012249229087196, total=14.1min
[CV] clsfr__estimator__kernel=poly, pl_NLP__pl_Txt__vectr__max_df=0.9 


C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



[CV]  clsfr__estimator__kernel=poly, pl_NLP__pl_Txt__vectr__max_df=0.9, score=0.3690694132659877, total=32.8min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 150.6min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('pl_NLP', FeatureUnion(n_jobs=None,
       transformer_list=[('pl_Txt', Pipeline(memory=None,
     steps=[('vectr', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...ity=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clsfr__estimator__kernel': ['rbf', 'poly'], 'pl_NLP__pl_Txt__vectr__max_df': [1.0, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_weighted', verbose=4)

In [209]:
def evaluate_model(model, X_test, Y_test, cat_names):
    """Evaluate the trained model and display classification reports.
        Args:
            X_test, Y_test : pd.DataFrame or np.array
                Testing sets of predictors (messages) and labels.
            cat_names: list of str
                Names of the classification categories.
        Returns:
            Y_pred: pd.DataFrame
                Predicted labels.
    """

    Y_pred = pd.DataFrame(model.predict(X_test), columns = cat_names)
    
    for i, cat in enumerate(cat_names):
        print('Classification report for category #{:d} - {}'.format(i, cat))
        print(classification_report(Y_test[cat], Y_pred[cat]))
    
    return Y_pred

Y_pred = evaluate_model(model, X_test_txt, Y_test, cat_names)




Classification report for category #0 - related
              precision    recall  f1-score   support

           0       0.12      0.01      0.01       153
           1       0.77      0.99      0.86       502

   micro avg       0.76      0.76      0.76       655
   macro avg       0.45      0.50      0.44       655
weighted avg       0.62      0.76      0.66       655

Classification report for category #1 - request
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       535
           1       0.68      0.12      0.21       120

   micro avg       0.83      0.83      0.83       655
   macro avg       0.76      0.56      0.56       655
weighted avg       0.81      0.83      0.78       655

Classification report for category #2 - offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       651
           1       0.00      0.00      0.00         4

   micro avg       0.99      0.99      0

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [207]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.77      0.99      0.86       502
           1       0.68      0.12      0.21       120
           2       0.00      0.00      0.00         4
           3       0.75      0.34      0.47       288
           4       0.74      0.23      0.35        62
           5       0.44      0.14      0.21        29
           6       1.00      0.04      0.08        23
           7       0.00      0.00      0.00        14
           8       0.50      0.12      0.19        17
           9       0.56      0.20      0.29        51
          10       0.68      0.20      0.31        74
          11       0.71      0.18      0.29        65
          12       1.00      0.11      0.20         9
          13       0.00      0.00      0.00        12
          14       0.00      0.00      0.00         7
          15       1.00      0.13      0.23        23
          16       0.75      0.08      0.15        36
          17       0.29    

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.

C:\Users\yma80\AppData\Local\Continuum\anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.



In [145]:
# Count related and unrelated messages
counts = df[['related', 'request', 'offer']].sum()
counts['unrelated'] = len(df)-counts['related']
counts['related'] -= (counts.request+counts.offer)

In [159]:
df[['weather_related', 'aid_related', 'infrastructure_related']].sum(axis=1).value_counts()

0    12177
1     9059
2     4059
3      885
dtype: int64

In [155]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Number of related or unrelated messages
n_related = df.related.value_counts().rename(index={0:'Unrelated', 1:'Related'})

fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
                    specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Sunburst(
    labels=["Total", "Related", "Unrelated", "Request", "Offer"],
    parents=["", "Total", "Total", "Related", "Related"],
    values=[0, counts.related, counts.unrelated, counts.request, counts.offer],
    hoverinfo='label+percent parent'
#     hovertemplate='<b>%{label} </b> <br> Sales: %{value}<br> Success rate: %{percent root:.2f}%',
),
              1, 1)

fig.add_trace(go.Pie(values=n_related.values, labels=n_related.index,
                     textinfo='label+percent', hole=0.4),
              row=1, col=2)

fig.update_layout(showlegend=True)
fig.show()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [151]:
X, Y, cat_names = load_data(sample_frac=0.1)
# Y = Y.iloc[:, 0]
# print(Y.value_counts())
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.25)

# train classifier
pipeline.fit(X_train, Y_train)

# print(Y.shape)

# predict on test data
Y_pred = pipeline.predict(X_test)

# # display results
# display_results(Y_test, Y_pred)

In [78]:
# train only the word processor
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# Building the classification part of the pipeline
clsfr = MultiOutputClassifier(LinearSVC())

clsfr.fit(X_train, Y_train)
Y_pred = clsfr.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [201]:
for i in range(4):
    print(classification_report(Y_test.values[:,i], Y_pred[:,i]))

IndexError: too many indices for array

### 6. Improve your model
Use grid search to find better parameters. 

In [147]:
parameters = {'vectr__max_df' : [1.0, 0.8, 0.7, 0.6],
              'vectr__min_df' : [0.0, 0.01]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_weighted', cv=3, verbose=4)

cv.fit(X_train, Y_train)

model = cv.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7127898967590667, total=  16.4s
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.8s remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7212080304774412, total=  16.9s
[CV] vectr__max_df=1.0, vectr__min_df=0.0 ............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   55.9s remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.0, score=0.7413184335905869, total=  16.8s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.4min remaining:    0.0s


[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7133483865761051, total=  16.8s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7265031659935167, total=  16.9s
[CV] vectr__max_df=1.0, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=1.0, vectr__min_df=0.01, score=0.7215772256285989, total=  16.9s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7127898967590667, total=  17.3s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7212080304774412, total=  17.8s
[CV] vectr__max_df=0.8, vectr__min_df=0.0 ............................
[CV]  vectr__max_df=0.8, vectr__min_df=0.0, score=0.7413184335905869, total=  17.2s
[CV] vectr__max_df=0.8, vectr__min_df=0.01 ...........................
[CV]  vectr__max_df=0.8, vectr__min_df=0.01, score=0.71334838657610

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 11.5min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [148]:
Y_pred = model.predict(X_test)
model.get_params()

{'clsfr': MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
            n_jobs=None),
 'clsfr__estimator': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'clsfr__estimator__C': 1.0,
 'clsfr__estimator__class_weight': None,
 'clsfr__estimator__dual': True,
 'clsfr__estimator__fit_intercept': True,
 'clsfr__estimator__intercept_scaling': 1,
 'clsfr__estimator__loss': 'squared_hinge',
 'clsfr__estimator__max_iter': 1000,
 'clsfr__estimator__multi_class': 'ovr',
 'clsfr__estimator__penalty': 'l2',
 'clsfr__estimator__random_state': None,
 'clsfr__estimator__tol': 0.0001,
 'clsfr__estimator__verbose': 0,
 'clsfr__n_jobs': N

### 9. Export your model as a pickle file